# Continous Growth - Chordia Method

Attempting to compute changes in Market Liquidity with the same variables as Chordia use.

The model is as follows:
                        DLjt = @ + DLMt + DLMt-1 + DLMt+1 + DLjt^2 + e

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.stats.stattools import durbin_watson

In [2]:
pd.set_option('use_inf_as_na', True)

In [3]:
df=pd.read_excel("BBREIT-1.07.xlsx",sheet_name="PROPORTIONAL_BID_ASK_SPREAD", index_col="Dates")
df.dropna(axis=1,inplace=True)

In [4]:
df

,AMT US Equity,PLD US Equity,CCI US Equity,EQIX US Equity,PSA US Equity,O US Equity,SBAC US Equity,WELL US Equity,VICI US Equity,DLR US Equity,...,CLPR US Equity,VOC US Equity,FREVS US Equity,AAIC US Equity,SELF US Equity,PW US Equity,SOHO US Equity,CORR US Equity,PEI US Equity,WHLR US Equity
Dates,,,,,,,,,,,,,,,,,,,,,
2020-08-04,0.000192,0.000095,0.000060,0.000895,0.000050,0.000162,0.000870,0.000188,0.000440,0.000062,...,0.003130,0.005602,0.033898,0.003752,0.005236,0.007507,0.013857,0.001082,0.009569,0.016713
2020-08-05,0.000353,0.000095,0.000365,0.000529,0.000508,0.000963,0.001208,0.002260,0.000447,0.000125,...,0.003175,0.011299,0.033898,0.003591,0.013072,0.057373,0.009217,0.001053,0.009569,0.010753
2020-08-06,0.000744,0.000095,0.000366,0.000493,0.000765,0.000161,0.000228,0.000177,0.000440,0.000064,...,0.001537,0.011905,0.033898,0.003656,0.031088,0.017939,0.017778,0.006270,0.009390,0.005038
2020-08-07,0.000039,0.000285,0.000059,0.000657,0.000050,0.000477,0.000546,0.000175,0.001748,0.000063,...,0.002967,0.040115,0.033898,0.003604,0.002594,0.021964,0.008696,0.000984,0.009479,0.013423
2020-08-10,0.000671,0.000385,0.000181,0.000447,0.000049,0.000319,0.000499,0.000174,0.000432,0.000379,...,0.004264,0.022599,0.033898,0.003527,0.012804,0.015131,0.012685,0.000908,0.007968,0.025316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-20,0.000042,0.000360,0.000372,0.000292,0.000033,0.000158,0.000529,0.000144,0.000303,0.000091,...,0.002519,0.008811,0.032749,0.003231,0.016231,0.009125,0.009524,0.009524,0.015190,0.013514
2022-09-21,0.000299,0.000092,0.000505,0.000661,0.000033,0.000160,0.000666,0.000145,0.000312,0.000092,...,0.001249,0.007168,0.014556,0.003252,0.024691,0.004158,0.009804,0.014528,0.014184,0.006826
2022-09-22,0.000516,0.000092,0.000255,0.000820,0.000770,0.000160,0.000440,0.000293,0.000318,0.000093,...,0.002594,0.007189,0.011834,0.003350,0.008873,0.025119,0.015748,0.010050,0.015666,0.014184


In [5]:
for i in df:
    df[i]= np.log(df[i])-np.log(df[i]).shift(1)

C:\Users\ravn_\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\ravn_\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
df.drop(index=df.index[0], axis=0, inplace=True)

In [9]:
df.isnull().values.sum()

55

In [10]:
df.dropna(inplace=True)

In [11]:
df.isnull().values.sum()

0

In [13]:
df["Sum"]=df.sum(axis=1)

In [14]:
Coefficients = []
Pvalues = []
Rsquared_adj = []
Bpagan = []
DBWatson = []
STD = []

for i in df:
    df["VOL"]= ((df[i]-df[i].shift(1))/df[i].shift(1))**2 # Squared return variable as we spoke about
    df["M"]=((df["Sum"]-df[i])/157) # Cross Sectional Mean Variable
    df["Mlag"]=df["M"].shift(1)
    df["Mlead"]=df["M"].shift(-1)
    df.replace([np.inf], np.nan, inplace=True)
    df=df.fillna(0)
        
        
      
    y = df[i]
    x = df[["M","VOL","Mlag","Mlead"]]
    x = sm.add_constant(x)
    model = sm.OLS(y,x, missing = 'drop') 
    results = model.fit()
    print(results.summary())
    print(sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x))
    print(durbin_watson(results.resid))
    
    
    # Filling the lists with the content specified
    coefficients=results.params 
    Coefficients.append(coefficients)
    
    pvalues=results.pvalues
    Pvalues.append(pvalues)
    
    rsquared_adj = results.rsquared_adj
    Rsquared_adj.append(rsquared_adj)
    
    bpagan = sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x)
    Bpagan.append(bpagan)
    
    std=results.bse
    STD.append(std)
    
    dbwatson = durbin_watson(results.resid)
    DBWatson.append(dbwatson)
    
    # CLearing the dynamic variables after use. Mostly for practical reuse-reasons
    Drop=df[["M","VOL","Mlag","Mlead"]]
    df.drop(columns=Drop, inplace=True)

                            OLS Regression Results                            
Dep. Variable:          AMT US Equity   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     12.89
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           5.27e-10
Time:                        11:08:24   Log-Likelihood:                -953.35
No. Observations:                 516   AIC:                             1917.
Df Residuals:                     511   BIC:                             1938.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0115      0.068     -0.168      0.8

(3.7403528621585362, 0.4422840620749099, 0.9327888324027495, 0.4445200654158695)
2.842610047784069
                            OLS Regression Results                            
Dep. Variable:          ARE US Equity   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     15.59
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           4.81e-12
Time:                        11:08:24   Log-Likelihood:                -933.79
No. Observations:                 516   AIC:                             1878.
Df Residuals:                     511   BIC:                             1899.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

(2.5090677572674296, 0.6430128099692312, 0.6242240823824265, 0.6454088958482755)
2.9179705857200338
                            OLS Regression Results                            
Dep. Variable:          WPC US Equity   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     14.79
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           1.93e-11
Time:                        11:08:24   Log-Likelihood:                -754.71
No. Observations:                 516   AIC:                             1519.
Df Residuals:                     511   BIC:                             1541.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

(20.665529168526632, 0.0003688699063823794, 5.3297751453633175, 0.00032738829855899173)
2.8374637954577926
                            OLS Regression Results                            
Dep. Variable:          HST US Equity   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.662
Method:                 Least Squares   F-statistic:                     253.5
Date:                Sat, 22 Oct 2022   Prob (F-statistic):          8.21e-120
Time:                        11:08:24   Log-Likelihood:                 247.97
No. Observations:                 516   AIC:                            -485.9
Df Residuals:                     511   BIC:                            -464.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------

(4.143179626761778, 0.38697525394139654, 1.0340610425643297, 0.38900687562598724)
2.8844780427998673
                            OLS Regression Results                            
Dep. Variable:          NNN US Equity   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.113
Method:                 Least Squares   F-statistic:                     17.47
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           1.91e-13
Time:                        11:08:24   Log-Likelihood:                -575.57
No. Observations:                 516   AIC:                             1161.
Df Residuals:                     511   BIC:                             1182.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

...

(4.820589833187746, 0.3062063238684809, 1.204724484088221, 0.30782240229001123)
2.769842562200747
                            OLS Regression Results                            
Dep. Variable:          INN US Equity   R-squared:                       0.273
Model:                            OLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                     47.95
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           3.08e-34
Time:                        11:08:26   Log-Likelihood:                -284.19
No. Observations:                 516   AIC:                             578.4
Df Residuals:                     511   BIC:                             599.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

(4.625902739023024, 0.3278785790397799, 1.155629661485573, 0.32961937762534377)
2.7299941011213567
                            OLS Regression Results                            
Dep. Variable:         NXDT US Equity   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     9.005
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           4.93e-07
Time:                        11:08:26   Log-Likelihood:                -672.42
No. Observations:                 516   AIC:                             1355.
Df Residuals:                     511   BIC:                             1376.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

(5.9928422993008175, 0.19968345293414355, 1.5011271747385289, 0.20055939091946098)
2.746361675416843
                            OLS Regression Results                            
Dep. Variable:           HT US Equity   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     35.08
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           6.74e-26
Time:                        11:08:27   Log-Likelihood:                -414.16
No. Observations:                 516   AIC:                             838.3
Df Residuals:                     511   BIC:                             859.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

(3.4779716890233843, 0.48123569313779313, 0.8669108033014116, 0.48357509576369695)
2.872006897171828
                            OLS Regression Results                            
Dep. Variable:           PW US Equity   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     3.069
Date:                Sat, 22 Oct 2022   Prob (F-statistic):             0.0162
Time:                        11:08:27   Log-Likelihood:                -578.76
No. Observations:                 516   AIC:                             1168.
Df Residuals:                     511   BIC:                             1189.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

C:\Users\ravn_\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


# Summary Results

Results look very good. COefficient higher than in previous models. Closer to one which is reasonable. R2 has a healthy value at around 16 per cent, p.value significance levels mean at around  half per cent and Breuch Pagan significance levels at around 40 per cent. Only test values that come about slightly negative is Durbin Watson with a mean test value at about 2.9. I will try to adjust the model a little with lagged values of dependent variable. However, all in all the results look very reasonable. 

In [16]:
Coef=pd.DataFrame(Coefficients)
round(Coef["M"].mean(),4)

0.9154

In [17]:
R2=pd.DataFrame(Rsquared_adj)
round(R2.mean(),4)

0    0.1617
dtype: float64

In [19]:
DBW=pd.DataFrame(DBWatson)
round(DBW.mean(),4)

0    2.8728
dtype: float64

In [24]:
PV=pd.DataFrame(Pvalues)
round(PV["M"].mean(),4)

0.0043

In [26]:
BP=pd.DataFrame(Bpagan)
round(BP[1].mean(),4)

0.4224